<a href="https://colab.research.google.com/github/tomasborrella/TheValley/blob/main/notebooks/spark02/Spark_DataFrames_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spark DataFrames

Notebook por [Tomás Borrella Martín](https://www.linkedin.com/in/tomasborrella/).

### Enlaces de interés
*   [Slides de presentación](https://docs.google.com/presentation/d/1MotclVSrLoykWogG-WwLa-DbPNvVgHBaGuZJX2Gfc4o/edit?usp=sharing)

# 1. Instalación Spark

In [1]:
# Install JAVA
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
# Install Spark
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz
!tar xf spark-3.1.1-bin-hadoop2.7.tgz

In [3]:
# Install findspark
!pip install -q findspark

In [4]:
# Environment variables
import os 
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

In [5]:
# Find spark
import findspark
findspark.init()

In [ ]:
# PySpark 
!pip install pyspark==3.1.1

# 2. Spark Session
Punto de entrada de la aplicación de Spark

In [7]:
# Imports
from pyspark.sql import SparkSession

In [8]:
# Create Spark Session
spark = (SparkSession
         .builder
         .master("local[*]")
         .appName("Spark Dataframes")
         .getOrCreate()
)

In [ ]:
# Show config
spark.sparkContext.getConf().getAll()

# Cargar datos

In [ ]:
# Descargamos los datos desde Github al entorno de Colab
!wget -P /content/data 'https://raw.githubusercontent.com/tomasborrella/TheValley/main/data/spark02/songs_log_small.json'

Leemos el archivo con `SparkSession.read`

Al leer un archivo externo con Spark crea un DataFrame

[Documentación oficial de read](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.SparkSession.read.html#pyspark.sql.SparkSession.read)

In [11]:
# Cargamos los datos en Spark
path = '/content/data/songs_log_small.json'
user_log = spark.read.json(path)

# Explorar datos

Comprobamos de qué tipo es el objeto que ha creado (DataFrame):

In [ ]:
type(user_log)

Mostramos información de su esquema con printSchema

[Documentación oficial de printSchema](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.printSchema.html#pyspark.sql.DataFrame.printSchema)

In [ ]:
user_log.printSchema()

El método `describe` calcula estadísticas básicas de las columnas. Es una transformación y por lo tanto es *lazy*

[Documentación oficial de describe](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.describe.html#pyspark.sql.DataFrame.describe)

In [ ]:
user_log.describe()

Es necesesario utilizar la acción `show` para que realice los cálculos y muestre el resultado

In [ ]:
user_log.describe().show()

La acción `show` muestra por defecto las primeras 20 líneas del dataframe, pero admite el número de líneas como parámetro

[Documentación oficial de show](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.show.html)

In [ ]:
user_log.show(n=3)

Se puede usar describe sobre una sola columna (o un subconjunto de ellas), para "seleccionar" la/s columna/s se utiliza `select`

[Documentación oficial de select](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.select.html#pyspark.sql.DataFrame.select)

In [ ]:
user_log.select('sessionId', 'ts').describe().show()

`select` es una transformación, y por lo tanto es *lazy*

In [ ]:
user_log.select('sessionId')

Como hemos visto previamente si queremos mostrar el contenido (20 primeras líneas) utilizaríamos show:



In [ ]:
user_log.select('sessionId').show()

Si queremos obtener todos los registros como una lista de filas utilizamos la acción `collect`

[Documentación oficial de collect](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.collect.html#pyspark.sql.DataFrame.collect)

In [ ]:
user_log.select('sessionId').collect()

Si el resultado lo queremos como una lista de Python podemos usar *List Comprehension*

In [34]:
mi_lista = [row[0] for row in user_log.select('sessionId').collect()]

In [ ]:
type(mi_lista)

In [ ]:
mi_lista

Si en lugar de todos los registros como una lista quisiéramos solo una parte de ellos podemos usar take

[Documentación oficial de take](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.take.html#pyspark.sql.DataFrame.take)

In [ ]:
user_log.select('sessionId').take(5)

Para contar el número de registros de un DataFrame se utiliza `count`

[Documentación oficial de count](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.count.html#pyspark.sql.DataFrame.count)

In [ ]:
user_log.count()

# Guardar datos

Para escribir datos se utiliza `write`

[Documentación oficial de write](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.write.html#pyspark.sql.DataFrame.write)

In [40]:
user_log.write.save('data/songs_log_small.csv', format='csv', header=True)

Después de escribirlos volvemos a leerlos para comprobarlos

In [42]:
user_log_2 = spark.read.csv('data/songs_log_small.csv', header=True)

In [ ]:
user_log_2.printSchema()

In [ ]:
user_log_2.take(2)

In [ ]:
user_log_2.count()

In [ ]:
user_log_2.select("userID").show()

In [ ]:
user_log_2.take(1)

Si comprobamos en el sistema de archivos podemos ver que tenemos tantos archivos como particiones tuviéramos (en este caso 2).

Si quisiéramos tener un único archivo, tendríamos que reparticionar los datos usando `coallesce(1)`

[Documentación oficial coallesce](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.coalesce.html#pyspark.sql.DataFrame.coalesce)

In [48]:
user_log.coalesce(1).write.save('data/songs_log_small_one.csv', format='csv', header=True)

# Spark Stop

In [49]:
spark.stop()